In [151]:
import math
import os
import PIL.Image as Image
import numpy as np

import torch

import torch.nn.functional as F
import torchvision



In [152]:
# Helper methods
def carla_k_matrix(fov=90.0, height=600, width=800):
    k = np.identity(3)
    k[0, 2] = width / 2.0
    k[1, 2] = height / 2.0
    k[0, 0] = k[1, 1] = width / \
                        (2.0 * math.tan(fov * math.pi / 360.0))
    return torch.from_numpy(k)

def read_depth(frame_num):
    input_path = os.path.abspath(f'./examples2/ForwardCamera0Depth/{str(frame_num).zfill(6)}.png')
    img = np.asarray(Image.open(input_path), dtype=np.uint8)
    img = img.astype(np.float64)  # .double()
    normalized_depth = np.dot(img, [1.0, 256.0, 65536.0])
    normalized_depth /= 16777215.0  # (256.0 * 256.0 * 256.0 - 1.0)
    #normalized_depth = torch.from_numpy(normalized_depth * 1000.0)
    normalized_depth = torch.from_numpy(normalized_depth * 1000.0)
    return normalized_depth.float()

def read_img(frame_num):
    input_path = os.path.abspath(f'./examples2/ForwardCamera0RGB/{str(frame_num).zfill(6)}.png')
    img = np.asarray(Image.open(input_path))
    return torch.from_numpy(img).float()


In [153]:
# lets load poses of two cameras, their depth and color images then warpe on to the other
def read_camera_poses(frame_num):
    rotations, translations = [], []
    with open('/home/habtegebrial/Desktop/Work/repos/PythonLibs/Carla_Script/examples2/ForwardCamera0.txt', 'r') as fid:
        for line in fid:
            data = np.array([float(p) for p in line.split(' ')]).reshape(3, 4)
            rotations.append(data[:, 0:3])
            translations.append(data[:, 3])
    rotations, translations =  np.array(rotations), np.array(translations)
    rotations, translations =  torch.from_numpy(rotations), torch.from_numpy(translations)
    return rotations[frame_num].view(3,3).float(), translations[frame_num].view(3, 1).float()

id_0, id_1 = 45, 46
depth_0, depth_1 = read_depth(id_0), read_depth(id_1)

img_0, img_1 = read_img(id_0).permute(2, 0, 1)/255.0, read_img(id_1).permute(2, 0, 1)/255.0
R_0, T_0 = read_camera_poses(id_0)
R_1, T_1 = read_camera_poses(id_1)


In [154]:
# compute camera 1 from camera 0
_, h, w = img_0.shape
print(img_0.shape)
x_locs = torch.linspace(0, w-1, w).view(1, w, 1).expand(h, w, 1)
y_locs = torch.linspace(0, h-1, h).view(h, 1, 1).expand(h, w, 1)
ones = torch.ones_like(x_locs)

hom_pts = torch.cat([x_locs, y_locs, ones], 2).view(h*w, 3, 1)

k_matrix = carla_k_matrix().float().view(1, 3, 3).expand(h*w, 3, 3)
k_inv = torch.inverse(carla_k_matrix().float()).view(1, 3, 3).expand(h*w, 3, 3)

# unproject
depth_vals = depth_1.view(h, w, 1).expand(h, w, 3).view(-1, 3, 1).float()
pts_3d = depth_vals * torch.bmm(k_inv, hom_pts).float()

# rotate from cam 1 to world
pts_3d_world = torch.bmm(R_1.view(-1, 3, 3).expand(h*w, 3, 3),
                        pts_3d) + T_1.view(-1, 3, 1).expand(h*w, 3, 1)

# world to camera 0
rot_w_0 = torch.inverse(R_0).view(-1, 3, 3).expand(h*w, 3, 3)
t_vec_w_0 = torch.mm(torch.inverse(R_0), (-1*T_0.view(3, 1)))
t_vec_w_0 = t_vec_w_0.view(-1, 3, 1).expand(h*w, 3, 1)
pts_3d_cam_0 = torch.bmm(rot_w_0,
                        pts_3d_world) + t_vec_w_0

pt3_cam_0_hom = torch.bmm(k_matrix, pts_3d_cam_0).view(-1, 3)

pt3_cam_0_hom[..., 0] /= pt3_cam_0_hom[..., 2]
pt3_cam_0_hom[..., 1] /= pt3_cam_0_hom[..., 2]

grid_pts_int = pt3_cam_0_hom[..., :-1]
grid_pts_int = grid_pts_int.view(h, w, 2)
h_w, h_h = (w-1)/2.0, (h-1)/2.0
grid_pts_int[..., 0] = (grid_pts_int[..., 0]-h_w)/h_w
grid_pts_int[..., 1] = (grid_pts_int[..., 1]-h_h)/h_h

# 
grid_pts_float = grid_pts_int.view(1, h, w, 2)
# input_img = img_0.view(1, 3, h, w)

warped_img = F.grid_sample(input=img_0.view(1, 3, h, w), grid=grid_pts_float)
print(torchvision.utils.save_image.__doc__)
torchvision.utils.save_image(filename='warped_img.png',
                             tensor=torchvision.utils.make_grid(warped_img))
torchvision.utils.save_image(filename='original.png', tensor=img_1)

# print(grid_pts_int.shape)
# print(img_1.max())
# print(warped_img.max())




torch.Size([3, 600, 800])
Save a given Tensor into an image file.

    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    


In [146]:
T_0

tensor([[381.7031],
        [326.2776],
        [ 39.6090]])

In [131]:
T_1

tensor([[368.8245],
        [326.2834],
        [ 39.5780]])